## Introductie

Deze les behandelt:
- Wat is het aanroepen van functies en waarvoor wordt het gebruikt
- Hoe maak je een functie-aanroep met OpenAI
- Hoe integreer je een functie-aanroep in een applicatie

## Leerdoelen

Na het afronden van deze les weet je hoe je:
- Het doel van het gebruik van functie-aanroepen begrijpt
- Een functie-aanroep opzet met de OpenAI Service
- Effectieve functie-aanroepen ontwerpt voor het gebruik in jouw applicatie


## Functie-aanroepen begrijpen

Voor deze les willen we een functie bouwen voor onze educatieve startup waarmee gebruikers via een chatbot technische cursussen kunnen vinden. We raden cursussen aan die passen bij hun vaardigheidsniveau, huidige functie en technologie waarin ze geïnteresseerd zijn.

Om dit te realiseren gebruiken we een combinatie van:
 - `OpenAI` om een chatervaring voor de gebruiker te creëren
 - `Microsoft Learn Catalog API` om gebruikers te helpen cursussen te vinden op basis van hun verzoek
 - `Function Calling` om de vraag van de gebruiker naar een functie te sturen die het API-verzoek uitvoert.

Laten we eerst kijken waarom we functie-aanroepen überhaupt zouden willen gebruiken:

print("Berichten in het volgende verzoek:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # haal een nieuw antwoord op van GPT waarbij het het antwoord van de functie kan zien


print(second_response.choices[0].message)


### Waarom Function Calling

Als je al een andere les in deze cursus hebt gevolgd, begrijp je waarschijnlijk de kracht van het gebruik van Large Language Models (LLMs). Hopelijk zie je ook enkele van hun beperkingen.

Function Calling is een functie van de OpenAI Service die is ontworpen om de volgende uitdagingen aan te pakken:

Inconsistente responsformattering:
- Voor function calling waren de antwoorden van een large language model ongestructureerd en inconsistent. Ontwikkelaars moesten complexe validatiecode schrijven om met elke variatie in de output om te gaan.

Beperkte integratie met externe data:
- Voor deze functie was het lastig om gegevens uit andere delen van een applicatie in een chatcontext te verwerken.

Door het standaardiseren van responsformaten en het mogelijk maken van naadloze integratie met externe data, maakt function calling de ontwikkeling eenvoudiger en vermindert het de noodzaak voor extra validatielogica.

Gebruikers konden geen antwoorden krijgen zoals "Wat is het huidige weer in Stockholm?". Dit komt doordat modellen beperkt waren tot de periode waarin de data getraind was.

Laten we hieronder een voorbeeld bekijken dat dit probleem illustreert:

Stel dat we een database willen maken met studentgegevens zodat we hen het juiste vak kunnen aanraden. Hieronder staan twee beschrijvingen van studenten die erg op elkaar lijken qua gegevens.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

We willen dit naar een LLM sturen om de gegevens te laten analyseren. Dit kan later in onze applicatie gebruikt worden om het naar een API te sturen of op te slaan in een database.

Laten we twee identieke prompts maken waarmee we de LLM instrueren over welke informatie we willen hebben:


We willen dit naar een LLM sturen om de onderdelen te analyseren die belangrijk zijn voor ons product. Zo kunnen we twee identieke prompts maken om de LLM te instrueren:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Na het aanmaken van deze twee prompts, sturen we ze naar de LLM met behulp van `openai.ChatCompletion`. We slaan de prompt op in de variabele `messages` en wijzen de rol toe aan `user`. Dit is om een bericht van een gebruiker aan een chatbot na te bootsen.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Hoewel de prompts hetzelfde zijn en de beschrijvingen op elkaar lijken, kunnen we verschillende formaten van de eigenschap `Grades` krijgen.

Als je de bovenstaande cel meerdere keren uitvoert, kan het formaat `3.7` of `3.7 GPA` zijn.

Dit komt doordat het LLM ongestructureerde data ontvangt in de vorm van de geschreven prompt en ook ongestructureerde data teruggeeft. We hebben een gestructureerd formaat nodig zodat we weten wat we kunnen verwachten bij het opslaan of gebruiken van deze data.

Door gebruik te maken van functioneel aanroepen, kunnen we ervoor zorgen dat we gestructureerde data terugkrijgen. Bij functioneel aanroepen voert het LLM eigenlijk geen functies uit. In plaats daarvan maken we een structuur waar het LLM zich aan moet houden bij het geven van antwoorden. Vervolgens gebruiken we die gestructureerde antwoorden om te bepalen welke functie we in onze applicaties moeten uitvoeren.


![Functie-aanroep stroomdiagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.nl.png)


### Gebruiksscenario's voor het gebruik van functie-aanroepen

**Externe tools aanroepen**  
Chatbots zijn erg goed in het geven van antwoorden op vragen van gebruikers. Door functie-aanroepen te gebruiken, kunnen chatbots berichten van gebruikers gebruiken om bepaalde taken uit te voeren. Bijvoorbeeld, een student kan de chatbot vragen: "Stuur een e-mail naar mijn docent dat ik meer hulp nodig heb bij dit vak." Dit kan een functie-aanroep doen naar `send_email(to: string, body: string)`

**API- of databasequeries maken**  
Gebruikers kunnen informatie vinden met natuurlijke taal die wordt omgezet in een geformatteerde query of API-verzoek. Een voorbeeld hiervan is een docent die vraagt: "Wie zijn de studenten die de laatste opdracht hebben afgerond?" Dit kan een functie aanroepen met de naam `get_completed(student_name: string, assignment: int, current_status: string)`

**Gestructureerde data creëren**  
Gebruikers kunnen een stuk tekst of een CSV-bestand nemen en het LLM gebruiken om belangrijke informatie eruit te halen. Bijvoorbeeld, een student kan een Wikipedia-artikel over vredesakkoorden omzetten om AI-flashcards te maken. Dit kan gedaan worden met een functie genaamd `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Je eerste functieaanroep maken

Het proces van het maken van een functieaanroep bestaat uit 3 hoofd stappen:
1. De Chat Completions API aanroepen met een lijst van je functies en een gebruikersbericht
2. De reactie van het model lezen om een actie uit te voeren, bijvoorbeeld een functie of API-aanroep uitvoeren
3. Nogmaals de Chat Completions API aanroepen met het antwoord van je functie om die informatie te gebruiken voor een reactie naar de gebruiker


![Stroom van een Functieaanroep](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.nl.png)


### Elementen van een functie-aanroep

#### Invoer van gebruikers

De eerste stap is het maken van een gebruikersbericht. Dit kan dynamisch worden toegekend door de waarde van een tekstveld te gebruiken, of je kunt hier een waarde instellen. Als je voor het eerst werkt met de Chat Completions API, moeten we de `role` en de `content` van het bericht definiëren.

De `role` kan `system` zijn (regels opstellen), `assistant` (het model) of `user` (de eindgebruiker). Voor het aanroepen van een functie stellen we dit in als `user` en geven we een voorbeeldvraag.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Functies maken.

Nu gaan we een functie definiëren en de parameters van die functie. We gebruiken hier slechts één functie genaamd `search_courses`, maar je kunt meerdere functies aanmaken.

**Belangrijk** : Functies worden opgenomen in het systeembericht naar de LLM en tellen mee voor het aantal beschikbare tokens dat je hebt.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definities**

De structuur van een functiedefinitie bestaat uit meerdere niveaus, elk met eigen eigenschappen. Hier volgt een overzicht van de geneste structuur:

**Eigenschappen van de functie op het hoogste niveau:**

`name` - De naam van de functie die we willen laten aanroepen.

`description` - Dit is de beschrijving van hoe de functie werkt. Het is hier belangrijk om specifiek en duidelijk te zijn.

`parameters` - Een lijst van waarden en het formaat die je wilt dat het model in zijn antwoord gebruikt.

**Eigenschappen van het parameters-object:**

`type` - Het gegevenstype van het parameters-object (meestal "object")

`properties` - Lijst van de specifieke waarden die het model zal gebruiken voor zijn antwoord

**Eigenschappen van individuele parameters:**

`name` - Impliciet gedefinieerd door de sleutel van de eigenschap (bijvoorbeeld "role", "product", "level")

`type` - Het gegevenstype van deze specifieke parameter (bijvoorbeeld "string", "number", "boolean")

`description` - Beschrijving van de specifieke parameter

**Optionele eigenschappen:**

`required` - Een array met daarin de parameters die verplicht zijn om de functieaanroep te voltooien


### De functie aanroepen
Na het definiëren van een functie, moeten we deze nu opnemen in het verzoek aan de Chat Completion API. Dit doen we door `functions` toe te voegen aan het verzoek. In dit geval is dat `functions=functions`.

Er is ook een optie om `function_call` op `auto` te zetten. Dit betekent dat we het LLM laten bepalen welke functie aangeroepen moet worden op basis van het bericht van de gebruiker, in plaats van dat we dit zelf toewijzen.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nu kijken we naar de reactie en hoe deze is opgemaakt:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Je ziet dat de naam van de functie wordt aangeroepen en dat het LLM uit het bericht van de gebruiker de gegevens kon halen om de argumenten van de functie in te vullen.


## 3. Functie-aanroepen integreren in een applicatie.

Nadat we de geformatteerde reactie van het LLM hebben getest, kunnen we deze nu integreren in een applicatie.

### Het proces beheren

Om dit in onze applicatie te integreren, volgen we de volgende stappen:

Als eerste doen we een aanvraag naar de OpenAI-diensten en slaan we het bericht op in een variabele genaamd `response_message`.


In [ ]:
response_message = response.choices[0].message

Nu zullen we de functie definiëren die de Microsoft Learn API aanroept om een lijst met cursussen op te halen:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Als best practice zullen we vervolgens kijken of het model een functie wil aanroepen. Daarna maken we een van de beschikbare functies aan en koppelen die aan de functie die wordt aangeroepen.  
Vervolgens nemen we de argumenten van de functie en koppelen die aan de argumenten van het LLM.

Tot slot voegen we het functie-aanroepbericht en de waarden die zijn teruggegeven door het `search_courses`-bericht toe. Dit geeft het LLM alle informatie die het nodig heeft om
de gebruiker in natuurlijke taal te antwoorden.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code-uitdaging

Goed gedaan! Om verder te leren over OpenAI Function Calling kun je het volgende bouwen: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Meer parameters van de functie toevoegen die leerlingen kunnen helpen om meer cursussen te vinden. Je vindt de beschikbare API-parameters hier:
 - Maak een andere functie-aanroep die meer informatie van de leerling gebruikt, zoals hun moedertaal
 - Voeg foutafhandeling toe wanneer de functie-aanroep en/of API-aanroep geen geschikte cursussen oplevert



---

**Disclaimer**:
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor kritische informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
